# Data Cleaning and Preparation

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

##  合并数据集

In [2]:
# 数据集的合并（merge）和 连接（join）是通过 一个 或多个 键 将行 连接起来的
df1 = DataFrame({'key':['b','b','a','c','a','a','b'],'data1':range(7)})
df2 = DataFrame({'key':['a','b','d'],'data2':range(3)})
df1
df2

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,key,data2
0,a,0
1,b,1
2,d,2


In [3]:
# 一种 多对一的合并
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [4]:
# 没有指明 要用 哪个列进行 行连接；如果没有进行指定，merge 就会 将 重叠列的列名 当做键
# 显示 指定 列名 进行连接
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [5]:
# 两个 待合并 的列名不同，也可 分别 进行指定
df3 = DataFrame({'lkey':['b','b','a','c','a','a','b'],'data1':range(7)})
df4 = DataFrame({'rkey':['a','b','d'],'data2':range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [6]:
# 注意 以上结果中，c、d结果消失了；默认情况下，merge 做的是 “insert”连接；（结果中的键是交集）
# merge 其他 连接：、
# left:左连接； right：右连接
# outer:外连接 求取的是 键的并集，组合了左连接 和 右连接的效果
pd.merge(df1, df2, how='outer')  # 确实 值 为 NaN
pd.merge(df1, df2, how='left')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


,key,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


In [7]:
# 多对多 连接 产生的 是 行的笛卡尔积
# 连接方式  只影响 出现在结果中的 键
df1 = DataFrame({'key':['b','b','a','c','a','b'], 'data1':range(6)})
df2 = DataFrame({'key':['a','b','a','b','d'], 'data2':range(5)})
# 因为 左边df 有 三个 ‘b’行，右边 有2个，所以结果中有 6 个 b行
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [8]:
# 进行交集 连接
pd.merge(df1,df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [9]:
# 根据 多个键 进行 合并，传入 一个由 列名 组成的列表即可
left = DataFrame({'key1':['foo','foo','bar'],'key2':['one','two','one'],'lval':[1,2,3]})
right = DataFrame({'key1':['foo','foo','bar','bar'],'key2':['one','one','one','two'], 'rval':[4,5,6,7]})
left
right
pd.merge(left, right, on=['key1','key2'], how='outer')

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [10]:
# merge 重复列名的处理
pd.merge(left, right, on='key1') # 注意 DataFrame left、right 有个重名  的key2 列
pd.merge(left, right, left_on='key1', right_on='key1', how='outer')  # left_on : 左侧 DataFrame 中 用于连接键的列
# 使用 merge 的 suffixes 选项 指定 附加到 左右 两个 DataFrame对象 重叠列名上的字符串
pd.merge(left, right, on='key1', suffixes=('_left','_right'))

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [11]:
# 依据 索引 进行 连接，使用 merge 的选项 left_index, right_index
left1 = DataFrame({'key':['a','b','a','a','b','c'], 'value':range(6)})
right1 = DataFrame({'group_val':[3.5, 7]}, index=['a','b'])
left1
right1
# 求 交集 inner
pd.merge(left1, right1, left_on='key', right_index=True)
# 求 键值 的并集  outer
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,group_val
a,3.5
b,7.0


,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [12]:
# 层次化 索引的处理 （较麻烦）
lefth = DataFrame({'key1':['Ohio','Ohio','Ohio','Nevada','Nevada'],
                  'key2':[2000,2001,2002,2001,2002],
                  'data':np.arange(5)})
righth = DataFrame(np.arange(12).reshape((6,2)),
                  index = [['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio'], 
                          [2001,2000,2000,2000,2001,2002]], columns=['event1','event2'])
lefth
righth

# 必须 指明 列表的形式 指明用作 合并键 的多个列（注意对 重复索引值的处理）
pd.merge(lefth, righth, left_on=['key1','key2'], right_index=True) # inner， 键 ： 交集
pd.merge(lefth, righth, left_on=['key1','key2'], right_index=True, how='outer') # 键 ： 并集

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [13]:
# 同时 使用 合并 双方的索引
left2 = DataFrame([[1.,2.], [3.,4.],[5.,6.]],
                 index=['a','c','e'],
                 columns=['Ohio','Nevada'])
right2 = DataFrame([[7.,8.],[9.,10.],[11.,12.],[13.,14.]],
                  index=['b','c','d','e'],
                  columns = ['Missouri','Alabama'])
left2
right2
# 将 两个 DataFrame 进行连接
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [14]:
# DataFrame中的  join实例方法-更容易地 实现 按照索引 合并，还可以用于合并 多个带有相同或者相似索引的DataFrame对象
# join 为 实例方法，调用方式：使用 实例对象.join()
left2.join(right2) # 默认 是 inner方法，键-交集
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
c,3.0,4.0,9.0,10.0
e,5.0,6.0,13.0,14.0


,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


### 轴向 连接

In [15]:
# 另一种数据操作 称作  连接（concatenation）、绑定（binding）或 堆叠（stacking）
# Numpy 有一个 用于 合并 原始 Numpy数组的 concatenation 函数
arr = np.arange(12).reshape((3,4))
arr
np.concatenate([arr, arr], axis=1)
# pandas 对象（如 Series和DataFrame），带有标签的轴 使 你 能够进一步推广 数组的连接运算
# 需要考虑 以下东西
# 1、如果 各 对象其他轴上的索引不通，那些轴 应该是 做 交集 or 并集
# 2、结果对象中的分组 需要各不相同嘛？
# 3、用于连接的轴 如何选择
# pandas 的 concat 函数
# 没有重复索引的三个 Series对象
s1 = Series([0,1], index=['a','b'])
s2 = Series([2,3,4], index=['c','d','e'])
s3 = Series([5,6], index=['f','g'])
s4 = pd.concat([s1 * 5, s3])
pd.concat([s1, s4], axis=1,sort=False)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [16]:
# 默认 情况下， concat是 在axis=0上工作的，产生 一个 新的 Series
pd.concat([s1,s2,s3])
# 指定 axis=1，结果就回变成一个 DataFrame
pd.concat([s1,s2,s3], axis=1,sort=True)

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [17]:
# 在 连接轴上创建 一个层次化索引，使用 选项 keys;
result = pd.concat([s1,s2,s3], keys=['one','two','three'])
result
result.unstack() # 将 数据的行 转化为 列
# 如果 沿着 axis=1 对 Series 进行合并 ， 则 keys 就会变成 DataFrame的列头
pd.concat([s1,s2,s3], axis=1, sort=False)
pd.concat([s1,s2,s3], axis=1, keys=['one','two','three'], sort=False) # keys 为 DataFrame的列头

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [18]:
# 对 DataFrame 逻辑相同
df1 = DataFrame(np.arange(6).reshape(3,2),
               index = ['a','b','c'],
               columns=['one','two'])
df2 = DataFrame(5 + np.arange(4).reshape(2,2),
               index=['a','c'],
               columns=['three','four'])
pd.concat([df1,df2], axis=1, keys=['level1','level2'],sort=False)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [19]:
# 若 concat函数 传入参数 非 列表，而是一个字典的话，则字典的key值就会被当做 keys选项的值
pd.concat({'level3':df1,'level4':df2}, axis=1,sort=False)
pd.concat({'level3':df1,'level4':df2}, axis=1,sort=False,
         names=['upper', 'lower'])

level3     level4     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

upper level3     level4     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [20]:
df1 = DataFrame(np.random.randn(3,4),
               columns=['a','b','c','d'])
df2 = DataFrame(np.random.rand(2,3),
               columns=['b','d','a'])
# 不保留 连接轴上的索引，产生一组 新索引 range(total_length),
pd.concat([df1, df2], ignore_index=True, sort=False)
pd.concat([df1, df2], sort=False)

,a,b,c,d
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221
3,0.964648,0.809813,NaN,0.872176
4,0.717454,0.723685,NaN,0.642475


,a,b,c,d
0,-0.204708,0.478943,-0.519439,-0.555730
1,1.965781,1.393406,0.092908,0.281746
2,0.769023,1.246435,1.007189,-1.296221
0,0.964648,0.809813,NaN,0.872176
1,0.717454,0.723685,NaN,0.642475


### 合并 重复数据

In [21]:
# 可能 索引 全部 或 部分重叠的 两个数据集，使用  NumPy的where函数，用于表达 一种矢量化的 if-else
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
          index=['f','e','d','c','b','a'])
b = Series(np.arange(len(a), dtype=np.float64),
           index=['f','e','d','c','b','a'])
a
b
# 如果 a 元素为 null， 那么选取 b的元素替换
np.where(pd.isnull(a), b, a)
# pd.combine_first 方法，实现同样的功能，而且 会 进行数据对齐
# combine_first 实例方法-两个对象合并，优先取第一个索引的值，而后再去取 第二个索引的值
# 可理解为： 用 参数对象中的数据  为 调用者对象  的缺失数据 “打补丁”
b[:-2].combine_first(a[2:])

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

array([0. , 2.5, 2. , 3.5, 4.5, 5. ])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [22]:
# combine_first() 可理解为： 用 参数对象中的数据  为 调用者对象  的缺失数据 “打补丁”
df1 = DataFrame({'a':[1.,np.nan, 5., np.nan],
                'b':[np.nan, 2.,np.nan,6.],
                'c':range(2, 18, 4)})
df2 = DataFrame({'a':[5., 4, np.nan, 3., 7.],
                'b':[np.nan,3.,4.,6.,8.]})
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 重塑 和 轴向旋转
用于 重新 排列 表格型数据的基础运算，称作 重塑（reshape）或 轴向旋转（pivot）运算

### 重塑层次化索引

默认情况下，stack和unstack操作的都是最内层（传入分层级别的编号 或 名称 即可对其他 级别进行 stack操作）

stack: 将数据的列 “旋转”为行（理解：旋转列头，作为一个层次索引）

unstack：将数据的行“旋转”为列

In [23]:
data = DataFrame(np.arange(6).reshape(2,3),
                index=pd.Index(['Ohio', 'Colorado'], name = 'state'),
                columns=pd.Index(['one','two','three'], name='number'))
data
# 理解：旋转 列头 转化为一个层次索引
result=data.stack()
result # 将 DataFrame转化为 pandas.core.series.Series

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [24]:
# 对于一个 层次化索引的 Series , 重新 将其 重新排列为一个 DataFrame
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [25]:
# 默认情况下，stack和unstack操作的都是最内层（传入分层级别的编号 或 名称 即可对其他 级别进行 unstack操作）
result.unstack(0) # result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [26]:
s1 = Series([0,1, 2, 3], index=['a','b','c','d'])
s2 = Series([4,5,6], index=['c','d','e'])
data2 = pd.concat([s1,s2],keys=['one', 'two'])
data2
data2.unstack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


## Handling Missing Data

In [27]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data
string_data.isnull()

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

0    False
1    False
2     True
3    False
dtype: bool

In [28]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### Filtering Out Missing Data

In [29]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data
data.dropna()

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

0    1.0
2    3.5
4    7.0
dtype: float64

In [30]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [32]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [6.5, NA, 3.]])
cleanedRow = data.dropna() # 返回 除去 包含 NA的行（默认 axis=0）
cleanedCol = data.dropna(axis=1) # 返回 除去 包含 NA的行（默认 axis=0）
data
cleanedRow
cleanedCol

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,6.5,NaN,3.0


,0,1,2
0,1.0,6.5,3.0


""
0
1
2
3


In [33]:
data.dropna(how='all') # 删除 同时满足 NA

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,6.5,NaN,3.0


In [34]:
data[4] = NA
data
data.dropna(axis=1, how='all')

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,6.5,NaN,3.0,NaN


,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,6.5,NaN,3.0


In [35]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df
df.dropna() # 默认 axis=0, 包含 1个 就 删除
# dropna 函数：thresh选项含义-应该是保留至少有n个 非NaN数据 的行/列
df.dropna(thresh=4,axis=1)

,0,1,2
0,-2.001637,NaN,NaN
1,-0.438570,NaN,NaN
2,3.248944,NaN,-0.577087
3,0.124121,NaN,0.523772
4,0.000940,1.343810,-0.713544
5,-0.831154,-2.370232,-1.860761
6,-0.860757,0.560145,-1.265934


,0,1,2
4,0.000940,1.343810,-0.713544
5,-0.831154,-2.370232,-1.860761
6,-0.860757,0.560145,-1.265934


,0,2
0,-2.001637,NaN
1,-0.438570,NaN
2,3.248944,-0.577087
3,0.124121,0.523772
4,0.000940,-0.713544
5,-0.831154,-1.860761
6,-0.860757,-1.265934


### Filling In Missing Data

In [36]:
df = pd.DataFrame(np.random.randn(7, 3), columns=['a','b','c'])
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df
df.fillna(0)

,a,b,c
0,0.119827,NaN,NaN
1,-2.359419,NaN,NaN
2,-0.970736,NaN,0.286350
3,0.377984,NaN,0.331286
4,1.349742,0.069877,0.246674
5,-0.011862,1.004812,1.327195
6,-0.919262,-1.549106,0.022185


,a,b,c
0,0.119827,0.000000,0.000000
1,-2.359419,0.000000,0.000000
2,-0.970736,0.000000,0.286350
3,0.377984,0.000000,0.331286
4,1.349742,0.069877,0.246674
5,-0.011862,1.004812,1.327195
6,-0.919262,-1.549106,0.022185


In [37]:
# 不同列 赋予 不通的值
df.fillna({1: 0.5, 2: 0})
df.fillna({'b': -0.5, 'c': 0})

,a,b,c
0,0.119827,NaN,NaN
1,-2.359419,NaN,NaN
2,-0.970736,NaN,0.286350
3,0.377984,NaN,0.331286
4,1.349742,0.069877,0.246674
5,-0.011862,1.004812,1.327195
6,-0.919262,-1.549106,0.022185


,a,b,c
0,0.119827,-0.500000,0.000000
1,-2.359419,-0.500000,0.000000
2,-0.970736,-0.500000,0.286350
3,0.377984,-0.500000,0.331286
4,1.349742,0.069877,0.246674
5,-0.011862,1.004812,1.327195
6,-0.919262,-1.549106,0.022185


In [38]:
# inplace 原地修改
_ = df.fillna(0, inplace=True)
df

,a,b,c
0,0.119827,0.000000,0.000000
1,-2.359419,0.000000,0.000000
2,-0.970736,0.000000,0.286350
3,0.377984,0.000000,0.331286
4,1.349742,0.069877,0.246674
5,-0.011862,1.004812,1.327195
6,-0.919262,-1.549106,0.022185


In [39]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df
df.fillna(method='ffill')  # 填充前面的值
df.fillna(method='ffill', limit=2) # 指定填充几个值

,0,1,2
0,0.758363,-0.660524,0.862580
1,-0.010032,0.050009,0.670216
2,0.852965,NaN,-0.023493
3,-2.304234,NaN,-1.218302
4,-1.332610,NaN,NaN
5,0.690002,NaN,NaN


,0,1,2
0,0.758363,-0.660524,0.862580
1,-0.010032,0.050009,0.670216
2,0.852965,0.050009,-0.023493
3,-2.304234,0.050009,-1.218302
4,-1.332610,0.050009,-1.218302
5,0.690002,0.050009,-1.218302


,0,1,2
0,0.758363,-0.660524,0.862580
1,-0.010032,0.050009,0.670216
2,0.852965,0.050009,-0.023493
3,-2.304234,0.050009,-1.218302
4,-1.332610,NaN,-1.218302
5,0.690002,NaN,-1.218302


In [40]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## Data Transformation（数据转化）
主要涉及 过滤、清理 以及 其他的转化工作

### Removing Duplicates（移除重复数据）

In [41]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [42]:
# DataFrame 的duplicated方法返回一个布尔值，表示各行是否有重复
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [43]:
# 用于 返回一个 移除了 重复行 的 DataFrame
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [44]:
data['v1'] = range(7)
# duplicated() 和 drop_duplicates()函数 默认会判断 全部列， 默认保留 的是第一次出现的 值组合
# 指定 部分列进行重复性判断， 如 只希望 根据 k1 列 过滤 重复项
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [45]:
data.drop_duplicates(['k1', 'k2'], keep='last') # keep ['first','last']

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping（利用函数 或 映射 进行数据转化）

In [46]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [47]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [48]:
lowercased = data['food'].str.lower()  # type(data['food'].str)  pandas.core.strings.StringMethods
lowercased
# Series 的 map 方法可以接受一个 函数 或 含有 映射关系 的字典型对象
data['animal'] = lowercased.map(meat_to_animal)
data

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [49]:
# 传入 一个 能够完成 全部 这些工作的函数
data['food'].map(lambda x: meat_to_animal[x.lower()])
# 使用 map 是一种 实现 元素级 转化 以及 其他数据清理工作的 便捷方式

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Replacing Values（替换值）

In [50]:
# 使用 fillna 方法 填充 缺失数据 可看作 值替换的一种 特殊情况
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [51]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [52]:
# 方式 一：多个 值 替换， -999 =》 np.nan, -1000 => np.nan
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [53]:
# 方式 二： 多个 值 替换， -999 =》 np.nan, -1000 => np.nan
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [54]:
data
# 方式 三： 多个 值 替换， -999 =》 np.nan, -1000 => np.nan
data.replace({-999: np.nan, -1000: 0})

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes（重命名轴索引）

In [55]:
# 跟 Series的值一样，轴标签 也可以 通过 函数或 映射 进行转化，从而得到一个新的对象
# 另外，轴还可以 被 就地转化，而无需新建一个 数据结构
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [56]:
# 跟 Series一样， 轴标签 也有一个 map方法
data.index.map(str.upper)
data
# 就地修改 （通过赋值）：将值 赋值给 index
data.index = data.index.map(str.lower)
data

Index(['OHIO', 'COLORADO', 'NEW YORK'], dtype='object')

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


,one,two,three,four
ohio,0,1,2,3
colorado,4,5,6,7
new york,8,9,10,11


In [57]:
# 取 每个 index 值的 前四个字符 并转化为 大写形式
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [58]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [59]:
# 数据集的转化版： 若 想要 想要数据集的转化版（而不是修改原始数据），比较实用的是rename
data.rename(index=str.title, columns=str.upper)
data

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [60]:
# rename 可以 结合字典型 对象 实现对 部分 轴标签  的更新
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [61]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning（离散化和面元划分）

In [62]:
# cut()和 qcut() 这两个 离散化函数 对 分量和分组分析非常重要
# 实现：划分为不通的年龄组
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [63]:
bins = [18, 25, 35, 60, 100]
# 传入 参数为 bins 为数组：组合为各个范围
cats = pd.cut(ages, bins)
# 返回值是一个特殊的 Categorical对象，可以看作一组表示 面元 名称 的字符串；
cats # pandas.core.arrays.categorical.Categorical

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [64]:
cats.codes # 代表 在 Categories 中 数据 在 bins 范围 数据的 下标；
cats.categories
pd.value_counts(cats) # 进行 值的统计

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [65]:
# 区间 可以通过 如下进行设置（区间 必须是有序的）
pd.cut(ages, [18, 26, 36, 61, 100], right=False)
binsTest = [18, 26, 36, 61, 100]
binsTest.reverse()
print(binsTest)
# pd.cut(ages, bins, right=False) # bins 必须是 升序的数组，要求 np.diff(bins) > 0 
# 源码 中， 如果 (np.diff(bins) < 0).any() : raise ValueError('bins must increase monotonically.')

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

[100, 61, 36, 26, 18]


In [66]:
# labels: 设置 自己 面元的名称，使用 选项 labels 设置为 一个列表或数组即可
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [67]:
# 均匀划分为 几组， 如果 pd.cut函数 bins的位置 为一个 数字
data = np.random.rand(20)
# 向 cut传入的 是面元的数量 而 非 确切的 面元边界（非数组），则会根据 数据的最小值 和最大值 计算 等长面元
# 将 均匀 分布 的数据 分成 四组
pd.cut(data, 4, precision=2)

[(0.72, 0.96], (0.02, 0.26], (0.02, 0.26], (0.26, 0.49], (0.49, 0.72], ..., (0.26, 0.49], (0.02, 0.26], (0.72, 0.96], (0.49, 0.72], (0.49, 0.72]]
Length: 20
Categories (4, interval[float64]): [(0.02, 0.26] < (0.26, 0.49] < (0.49, 0.72] < (0.72, 0.96]]

In [68]:
# qcut类似cut函数，它可以根据 样本 分位数 对数据进行 面元划分；得到大小基本相等的 面元
data = np.random.randn(1000)  # Normally distributed（正态分布）
cats = pd.qcut(data, 4)  # Cut into quartiles 按照四分数 进行切割
cats
pd.value_counts(cats)

[(-0.686, -0.0453], (0.604, 3.928], (-0.0453, 0.604], (0.604, 3.928], (-0.0453, 0.604], ..., (-0.0453, 0.604], (-2.9499999999999997, -0.686], (-0.0453, 0.604], (0.604, 3.928], (-0.686, -0.0453]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.686] < (-0.686, -0.0453] < (-0.0453, 0.604] < (0.604, 3.928]]

(0.604, 3.928]                   250
(-0.0453, 0.604]                 250
(-0.686, -0.0453]                250
(-2.9499999999999997, -0.686]    250
dtype: int64

In [69]:
# 自定义 分位数（0到1之间的数值，包含端点）
cats1 = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])
pd.value_counts(cats1)

(-0.0453, 1.287]                 400
(-1.191, -0.0453]                400
(1.287, 3.928]                   100
(-2.9499999999999997, -1.191]    100
dtype: int64

### Detecting and Filtering Outliers（检查和过滤异常值）
异常值：亦称为 孤立点 或 离群值

In [70]:
data = pd.DataFrame(np.random.randn(1000, 4))
data
data.describe()

,0,1,2,3
0,1.832492,1.658668,0.454105,-0.194964
1,-0.976914,-0.296088,-0.581274,-0.019941
2,-0.038100,1.192630,-1.098127,-0.632915
3,0.802414,-0.065061,-0.154208,0.921908
4,1.956463,0.379706,0.270780,-1.127073
5,0.525962,0.590275,1.139096,0.054854
6,0.550078,0.160381,0.160301,-0.712900
7,-0.308272,0.317635,2.458842,-0.878777
8,0.850684,0.205855,0.303005,-0.225959
9,1.905455,1.496694,0.133961,-0.503657


,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.041946,0.048124,0.026286,-0.010029
std,1.000010,1.000482,1.009975,0.992731
min,-3.428254,-3.645860,-3.184377,-3.745356
25%,-0.740152,-0.599807,-0.612162,-0.699863
50%,-0.085000,0.047101,-0.008168,-0.031732
75%,0.629197,0.750195,0.690847,0.692355
max,3.366626,2.653656,3.525865,2.735527


In [71]:
# 找出 第二列 中 绝对值 大于 3的值
col = data[2]
col[np.abs(col) > 3]

51     3.260383
226   -3.056990
313   -3.184377
773    3.525865
Name: 2, dtype: float64

In [72]:
# 选出 全部 含有 “超过 3或 -3 的值”的行
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
32,-2.315555,0.457246,-0.025907,-3.399312
51,0.050188,1.951312,3.260383,0.963301
127,0.146326,0.508391,-0.196713,-3.745356
226,-0.293333,-0.242459,-3.056990,1.918403
250,-3.428254,-0.296336,-0.439938,-0.867165
313,0.275144,1.179227,-3.184377,1.369891
535,-0.362528,-3.548824,1.553205,-2.186301
627,3.366626,-2.372214,0.851010,1.332846
773,-0.658090,-0.207434,3.525865,0.283070
794,0.599947,-3.645860,0.255475,-0.549574


In [73]:
# 根据条件进行 赋值操作
# np.sign() 这个 ufunc 返回的是一个 由 1和 -1 组成的数组，表示原始符号
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.041885,0.049319,0.025742,-0.008884
std,0.997463,0.996470,1.006739,0.988912
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.740152,-0.599807,-0.612162,-0.699863
50%,-0.085000,0.047101,-0.008168,-0.031732
75%,0.629197,0.750195,0.690847,0.692355
max,3.000000,2.653656,3.000000,2.735527


In [74]:
np.sign(data).head()

,0,1,2,3
0,1.0,1.0,1.0,-1.0
1,-1.0,-1.0,-1.0,-1.0
2,-1.0,1.0,-1.0,-1.0
3,1.0,-1.0,-1.0,1.0
4,1.0,1.0,1.0,-1.0


### Permutation and Random Sampling（排列和随机采样）

In [75]:
# numpy.random.permutation 函数 可以轻松实现对 Series或 DataFrame的列 排列 工作
# 通过需要 排列 的轴 的长度，调用 permutation(),可以产生 一个 表示 新顺序 的 整数数组
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
# np.random.permutation(n)：产生 [0,n) 数字的一个随机排列（注意区间）
sampler = np.random.permutation(5)
sampler

array([2, 3, 4, 0, 1])

In [76]:
df
# 就 可以 使用 基于 ix的索引操作  或 take函数中使用该数组了（take应该是取index的数据行）
df.take(sampler)

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19
0,0,1,2,3
1,4,5,6,7


In [77]:
# 实现 非替换式采样 （即，用非替换的方式  选取随机子集）
# 使用 permutation函数：从permutation返回的数组中 切下 前 k个元素（其中 k为期望的自己大小）
df.take(np.random.permutation(len(df))[:3])  # 随机 选取 df 所有行的 随机三行

,0,1,2,3
0,0,1,2,3
4,16,17,18,19
2,8,9,10,11


In [78]:
# 非 替换 式 采样，直接使用sample方法
df.sample(n=3)

,0,1,2,3
2,8,9,10,11
3,12,13,14,15
0,0,1,2,3


In [79]:
# 替换式 采样（替换的方式产生样本）
# 通过 np.random.randint得到 一组 随机整数
bag = np.array([5,7,-1.,6,4])
sampler = np.random.randint(0, len(bag), size=10)
sampler
draws = bag.take(sampler)
draws
# 或 使用 sample 函数
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

array([0, 2, 3, 2, 2, 1, 1, 1, 0, 4])

array([ 5., -1.,  6., -1., -1.,  7.,  7.,  7.,  5.,  4.])

0    5
3    6
2   -1
2   -1
1    7
0    5
0    5
2   -1
0    5
2   -1
dtype: int64

### Computing Indicator/Dummy Variables（计算 指标 / 哑变量）
    另一种常用于统计建模 或 机器学习的转化方式：将 分类变量（categorical variable） 转化为 “哑变量矩阵”（dummy matrix） 或 “指标矩阵” （indicator matrix）
    如果 DataFrame的某一列中 含有 K个不同的值，则可以派生出 一个 k列矩阵或 DataFrame(其值 全为 1和0）；
    pandas有一个 get_dummies函数 可以实现此功能


In [80]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df
# get_dummies：就是添加原来数据中没有的变量，但是这并不是意味着可以随意添加，应该是根据原来的数据进行转换
# 如下 就是表示: 字母_a-这个位置是否是a; 字母_b-这个位置是不是b;
pd.get_dummies(df['key'], prefix='字母')

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


,字母_a,字母_b,字母_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [81]:
# get_dummies函数的 prefix参数 给 列上加上一个前缀
dummies = pd.get_dummies(df['key'], prefix='key')
dummies
# type(df['data1'])  # pandas.core.series.Series
# type(df[['data1']]) # pandas.core.frame.DataFrame
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [82]:
!cat datasets/movielens/movies.dat

1::Toy Story (1995)::Animation|Children's|Comedy
2::Jumanji (1995)::Adventure|Children's|Fantasy
3::Grumpier Old Men (1995)::Comedy|Romance
4::Waiting to Exhale (1995)::Comedy|Drama
5::Father of the Bride Part II (1995)::Comedy
6::Heat (1995)::Action|Crime|Thriller
7::Sabrina (1995)::Comedy|Romance
8::Tom and Huck (1995)::Adventure|Children's
9::Sudden Death (1995)::Action
10::GoldenEye (1995)::Action|Adventure|Thriller
11::American President, The (1995)::Comedy|Drama|Romance
12::Dracula: Dead and Loving It (1995)::Comedy|Horror
13::Balto (1995)::Animation|Children's
14::Nixon (1995)::Drama
15::Cutthroat Island (1995)::Action|Adventure|Romance
16::Casino (1995)::Drama|Thriller
17::Sense and Sensibility (1995)::Drama|Romance
18::Four Rooms (1995)::Thriller
19::Ace Ventura: When Nature Calls (1995)::Comedy
20::Money Train (1995)::Action
21::Get Shorty (1995)::Action|Comedy|Drama
22::Copycat (1995)::Crime|Drama|Thriller
23::Assassins (1995)::Thriller
24::Powder (1995)::Drama|Sci-Fi
25::Le

In [83]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10]

/home/zhang/software/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [84]:
# 使用 pd.unique进行去重
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)

In [85]:
genres
# 第二种方式（巧用 set.union）: 从数据集 中 抽取出 不同 的 genre值
genre_iter = (set(x.split('|')) for x in movies.genres)
genres2 = sorted(set.union(*genre_iter))
genres2

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [86]:
# 从 一个全零 的 DataFrame 开始构建 指标DataFrame
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [87]:
# 样例 定位 第一行 数据 genres 的 每个元素 在 指标 matrix 中的 位置
gen = movies.genres[0]
gen.split('|')
# gen位于 指标DataFrame的 列号， 即确定 指标 DataFrame 中的值 （即 位置）
dummies.columns.get_indexer(gen.split('|'))

['Animation', "Children's", 'Comedy']

array([0, 1, 2])

In [88]:
# 迭代 每一部 电影 并将 dummies 各行 的项 设置为1
for i, gen in enumerate(movies.genres): # i ： 代表 index
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1  # 定位 ， 赋值
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
# 再 将其 与 movies（DataFrame对象） 合并起来
# DataFrame.add_prefix('Genre_')  # 为 DataFrame对象 的列 添加 前缀
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

## 注意此种方案，构建 多成员指标变量 会变得 非常慢，肯定需要编写一个能够利用 DataFrame内部机制的更低级函数才行

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
                                ...             
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

In [90]:
# 一个对 统计 有用的秘诀：结合 get_dummies 和 诸如 cut之类的 离散化 函数
# 统计 各个位置 是那个区域的值 
np.random.seed(12345)
values = np.random.rand(10)
values
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.cut(values, bins)
pd.get_dummies(pd.cut(values, bins))

array([0.9296, 0.3164, 0.1839, 0.2046, 0.5677, 0.5955, 0.9645, 0.6532,
       0.7489, 0.6536])

[(0.8, 1.0], (0.2, 0.4], (0.0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.4, 0.6], (0.8, 1.0], (0.6, 0.8], (0.6, 0.8], (0.6, 0.8]]
Categories (5, interval[float64]): [(0.0, 0.2] < (0.2, 0.4] < (0.4, 0.6] < (0.6, 0.8] < (0.8, 1.0]]

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## String Manipulation（字符串操作）

### String Object Methods（字符串对象方法）

In [91]:
# split函数 字符串拆分为数组
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

In [93]:
# 结合strip 修剪 空白符（包括换行符）
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [94]:
# 字符串 拼接
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [95]:
# join函数  拼接
'::'.join(pieces)

'a::b::guido'

In [98]:
# 子串 定位 问题
'guido' in val # 检查 子串的方式
val.index(',') 
val.find(':')

True

1

-1

In [102]:
# find方法  如果在字符串中 找到子串，则返回 第一个 发现的子串的 第一个字符所在的位置；如果没有找到则返回-1
# rfind方法 如果在字符串中 找到子串，则返回最后一个发现的子串的 第一个字符所在的位置；如果没有找到则返回-1
val.rfind(',')

3

In [99]:
# find vs index  函数 ： 若 待查找的 字符串 不存在， index 会 引发一个异常（而非返回 -1）
val.index(':')

ValueError: substring not found

In [100]:
# 统计 子串出现 次数
val.count(',')

2

In [101]:
# replace函数 用于 将 指定模式  替换为 另外一种模式，也常常用于删除模式：传入空字符串
val.replace(',', '::')
val.replace(',', '')

'a::b::  guido'

'ab  guido'

In [108]:
# startswith: 如果 字符串以 某个前缀开头，则返回 True
# endswith : 如果 字符串以 某个后缀结尾，则返回 True
# strip, rstrip,lstrip
# split
# lower,upper
# ljust,rjust：用 空格（或其他字符） 填充字符串的 空白侧 以返回 符合最低宽度的字符串
len(val)
val.ljust(13,'M')  # 返回一个原字符串左对齐,并使用空格填充至指定长度的新字符串。如果指定的长度小于原字符串的长度则返回原字符串。

11

'a,b,  guidoMM'

### Regular Expressions（正则表达式regex）
re模块的函数可以分为三个大类：模式匹配、替换以及拆分

In [109]:
import re
# 拆分 一个 字符串，分隔符 为 数量 不定的 一组 空白符（制表符、空格、换行符等）。regex: \s+
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)
# 调用 re.split() 函数时，正则表达式 会先 被编译，然后 再在 text 上 调用其 split方法

['foo', 'bar', 'baz', 'qux']

In [113]:
#  re.complie 自己编译 regex 以得到 一个可重用 的 regex对象
# 此种方式 优势：如果打算对许多字符串应用 同一条 表达式操作，可以节省大量的 CPU时间
regex = re.compile('\s+') # re.Pattern 对象
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [112]:
# 得到 re.Pattern对象（regex）的所有模式
regex.findall(text)

['    ', '\t ', '  \t']

In [114]:
# match和 search 跟 findall功能类似：
# findall 返回的是  字符串中 所有的匹配项，而 search则只返回第一个匹配项；
# match 更加 严格， 它 只匹配字符串的首部（只匹配 出现在 字符串 开头的模式）
# sub方法：它会将 匹配到的 模式 替换为 指定字符串，并返回 所得到的 新字符串
# 有一段文本 以及 一条能够 识别大部分电子邮件地址的 正则表达式
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [115]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [118]:
m = regex.search(text) # 返回 文本中 第一个电子邮件地址（以特殊的匹配项对象 形式 返回）
m  # <re.Match object; span=(5, 20), match='dave@google.com'>
text[m.start():m.end()]

<re.Match object; span=(5, 20), match='dave@google.com'>

'dave@google.com'

In [120]:
# 因为 regex.match 只匹配 出现在 字符串 开头的模式，返回 None
print(regex.match(text))

None


In [121]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [122]:
# 找出 电子邮件，并对 各个电子邮件地址 拆分为 3部分：用户名、域名以及 域名后缀
# 只需 将 待分段的模式的 各部分 用 圆括号包起来即可
#pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'  
regex = re.compile(pattern, flags=re.IGNORECASE)

In [126]:
m = regex.match('wesm@bright.net')
m
m.groups()

<re.Match object; span=(0, 15), match='wesm@bright.net'>

('wesm', 'bright', 'net')

In [128]:
# 对于 带有分组功能  的模式 ，对应  findall 会返回 一个元组列表
regex.findall(text)
regex.findall('wesm@bright.net')

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

[('wesm', 'bright', 'net')]

In [130]:
# sub 还能 通过 诸如 \1 \2之类的 特殊符号 访问个匹配项中的 分组；
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



### Vectorized String Functions in pandas（pandas中 矢量化的字符串函数）

In [143]:
# 含有 字符串的列 包含 一些缺失数据
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data
data.isnull()

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [144]:
# 通过 data.map, 所有 字符串和正则表达式方法都能被应用于（传入 lambda表达式或者其他函数）各个值，但是 如果存在 NA就会导致报警
# Series的str属性中，包含一些可以跳过 NA值的 字符串方法
# 通过 str.contains检查各个电子邮件地址是否含有 ‘gmail’
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [145]:
# 使用 正则表达式
pattern
data.str.findall(pattern, flags=re.IGNORECASE)

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [146]:
# 实现 矢量化的元素 获取操作：
# 方法1：使用 str.get 
# 方法2：在 str属性上使用索引
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches  # pandas.core.series.Series

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [147]:
# matches.str # <pandas.core.strings.StringMethods at 0x7f46ea5082b0>
matches.str.get(1)
matches.str[0]

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

In [149]:
# 对 字符串 进行 截取
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

In [150]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

## Conclusion